# 실습 05: OCR 신뢰도 측정 및 결과 시각화

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/main/notebooks/Lab05_신뢰도측정.ipynb)

## 🎯 학습 목표
- OCR 신뢰도(Confidence) 이해
- 저신뢰도 영역 탐지
- 품질 관리 자동화

## ⏱️ 소요 시간: 25분
## 📊 난이도: ⭐⭐⭐☆☆


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 다양한 품질의 이미지 생성
from PIL import Image, ImageDraw, ImageFilter
import cv2
import numpy as np

# 1. 깨끗한 이미지
img_clean = Image.new('RGB', (800, 300), color='white')
draw = ImageDraw.Draw(img_clean)
draw.text((20, 20), "고품질 텍스트: 계약금액 1,500,000원", fill='black')
img_clean.save('clean.jpg')

# 2. 노이즈 이미지
img_noisy = Image.new('RGB', (800, 300), color='white')
draw = ImageDraw.Draw(img_noisy)
draw.text((20, 20), "저품질 텍스트: 계약금액 1,500,000원", fill='black')
img_noisy = img_noisy.filter(ImageFilter.GaussianBlur(radius=2))
# 노이즈 추가
img_cv = cv2.cvtColor(np.array(img_noisy), cv2.COLOR_RGB2BGR)
noise = np.random.normal(0, 30, img_cv.shape).astype(np.uint8)
img_cv = cv2.add(img_cv, noise)
cv2.imwrite('noisy.jpg', img_cv)

print("✅ 테스트 이미지 생성 완료!")


In [ ]:
# Tesseract로 신뢰도 측정
import pytesseract
from pytesseract import Output
import pandas as pd

def analyze_confidence(image_path, label):
    img = cv2.imread(image_path)
    data = pytesseract.image_to_data(img, lang='kor+eng', output_type=Output.DICT)
    
    # 신뢰도 분석
    confidences = [int(conf) for conf in data['conf'] if int(conf) > 0]
    texts = [text for text in data['text'] if text.strip()]
    
    print(f"\n{'='*60}")
    print(f"📊 {label}")
    print(f"{'='*60}")
    print(f"  평균 신뢰도: {np.mean(confidences):.1f}%")
    print(f"  최소 신뢰도: {np.min(confidences):.1f}%")
    print(f"  최대 신뢰도: {np.max(confidences):.1f}%")
    print(f"  저신뢰도(<70%) 단어 수: {sum(1 for c in confidences if c < 70)}")
    print(f"\n  인식된 텍스트: {' '.join(texts)}")
    
    return confidences

conf_clean = analyze_confidence('clean.jpg', '깨끗한 이미지')
conf_noisy = analyze_confidence('noisy.jpg', '노이즈 이미지')


In [ ]:
# 신뢰도 분포 시각화
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 깨끗한 이미지
axes[0].hist(conf_clean, bins=20, color='green', alpha=0.7, edgecolor='black')
axes[0].axvline(70, color='red', linestyle='--', label='기준선(70%)')
axes[0].set_title('깨끗한 이미지 - 신뢰도 분포', fontsize=12)
axes[0].set_xlabel('신뢰도 (%)')
axes[0].set_ylabel('빈도')
axes[0].legend()
axes[0].grid(alpha=0.3)

# 노이즈 이미지
axes[1].hist(conf_noisy, bins=20, color='orange', alpha=0.7, edgecolor='black')
axes[1].axvline(70, color='red', linestyle='--', label='기준선(70%)')
axes[1].set_title('노이즈 이미지 - 신뢰도 분포', fontsize=12)
axes[1].set_xlabel('신뢰도 (%)')
axes[1].set_ylabel('빈도')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ 신뢰도 분석 완료!")


In [ ]:
# 품질 관리 자동화: 저신뢰도 경고
def quality_check(image_path, threshold=70):
    img = cv2.imread(image_path)
    data = pytesseract.image_to_data(img, lang='kor+eng', output_type=Output.DICT)
    
    low_conf_words = []
    for i, conf in enumerate(data['conf']):
        if int(conf) > 0 and int(conf) < threshold:
            text = data['text'][i]
            if text.strip():
                low_conf_words.append({
                    'text': text,
                    'confidence': int(conf),
                    'bbox': (data['left'][i], data['top'][i], 
                            data['width'][i], data['height'][i])
                })
    
    return low_conf_words

# 검사 실행
print("\n🔍 품질 검사 중...\n")
issues = quality_check('noisy.jpg', threshold=70)

if issues:
    print(f"⚠️  경고: {len(issues)}개의 저신뢰도 단어 발견!")
    print("\n저신뢰도 단어 목록:")
    for item in issues:
        print(f"  - '{item['text']}': {item['confidence']}%")
    print("\n➡️  재촬영 또는 전처리 필요")
else:
    print("✅ 품질 검사 통과!")
